In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-09-04 09:20:09--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2023-09-04 09:20:16 (10.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [6]:
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [12]:
chars = sorted(list(set(text)))
num_chars = len(chars)
print(''.join(chars))
print(f'vocab size: {num_chars}')


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [15]:
### Tokenize ###

# encoder - take a string (list of chars) and output a list of integers
stoi = { ch: i for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]

# decoder - take a list of integers and output a string
itos = { i: ch for i, ch in enumerate(chars) }
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello world"))
print(decode(encode("hello world")))


[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [17]:
# encode the training data

import torch
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:500])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [20]:
# Stlip into test/train
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

print(len(train_data))
print(len(test_data))

1003854
111540


In [21]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [23]:
torch.manual_seed(1337)

batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the max context length for predictions?

# generate a small batch of data of inputs x and targets y
def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # random offsets into dataset
    x = torch.stack([data[i:i+block_size] for i in ix ])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # y is offset by 1 from x
    return x, y
  
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [24]:
# spell out the x, y for understanding
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is: {context.tolist()} the target is: {target}')

when input is: [24] the target is:43
when input is: [24, 43] the target is:58
when input is: [24, 43, 58] the target is:5
when input is: [24, 43, 58, 5] the target is:57
when input is: [24, 43, 58, 5, 57] the target is:1
when input is: [24, 43, 58, 5, 57, 1] the target is:46
when input is: [24, 43, 58, 5, 57, 1, 46] the target is:43
when input is: [24, 43, 58, 5, 57, 1, 46, 43] the target is:39
when input is: [44] the target is:53
when input is: [44, 53] the target is:56
when input is: [44, 53, 56] the target is:1
when input is: [44, 53, 56, 1] the target is:58
when input is: [44, 53, 56, 1, 58] the target is:46
when input is: [44, 53, 56, 1, 58, 46] the target is:39
when input is: [44, 53, 56, 1, 58, 46, 39] the target is:58
when input is: [44, 53, 56, 1, 58, 46, 39, 58] the target is:1
when input is: [52] the target is:58
when input is: [52, 58] the target is:1
when input is: [52, 58, 1] the target is:58
when input is: [52, 58, 1, 58] the target is:46
when input is: [52, 58, 1, 58, 4